In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,047 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,169 

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder\
    .appName("SparkSQL")\
    .config("spark.sql.debug.maxToStringFields", 2000)\
    .config("spark.driver.memory", "2g")\
    .getOrCreate()

# Set the partitions to 4 or 8. 
spark.conf.set("spark.sql.shuffle.partitions", 8)


In [10]:
import pandas as pd

In [35]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
oscars_df = pd.read_csv("oscars_df.csv")
# Review the DataFrame
pd.set_option('display.max_columns', None)
oscars_df

,Unnamed: 0,Film,Oscar Year,Film Studio/Producer(s),Award,Year of Release,Movie Time,Movie Genre,IMDB Rating,IMDB Votes,Movie Info,Genres,Critic Consensus,Content Rating,Directors,Authors,Actors,Original Release Date,Streaming Release Date,Production Company,Tomatometer Status,Tomatometer Rating,Tomatometer Count,Audience Status,Audience Rating,Audience Count,Tomatometer Top Critics Count,Tomatometer Fresh Critics Count,Tomatometer Rotten Critics Count,Film ID
0,0,Wings,1927/28,Famous Players-Lasky,Winner,1927,144,"Drama,Romance,War",7.5,"12,221","With World War I afoot, David Armstrong (Richa...","Action & Adventure, Classics",Subsequent war epics may have borrowed heavily...,PG-13,William Wellman,"Hope Loring, Louis D. Lighton","Clara Bow, Charles 'Buddy' Rogers, Richard Arl...",1927-08-12,2017-01-02,Unknown,Certified-Fresh,93.0,46.0,Upright,78.0,3530.0,9.0,43.0,3.0,2becf7d5-a3de-46ab-ae45-abdd6b588067
1,1,7th Heaven,1927/28,Fox,Nominee,1927,110,"Drama,Romance",7.7,"3,439",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19ed3295-a878-4fd2-8e60-5cd7b5f93dad
2,2,The Racket,1927/28,The Caddo Company,Nominee,1928,84,"Crime,Drama,Film-Noir",6.7,"1,257",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3111c2d8-0908-4093-8ff3-99c89f2f2f08
3,3,The Broadway Melody,1928/29,Metro-Goldwyn-Mayer,Winner,1929,100,"Drama,Musical,Romance",5.7,"6,890","Vaudeville sisters ""Hank"" (Bessie Love) and Qu...","Musical & Performing Arts, Romance",The Broadway Melody is interesting as an examp...,NR,Harry Beaumont,"James Gleason, Norman Houston, Sarah Y. Mason","Anita Page, Bessie Love, Charles King, Jed Pro...",1929-02-01,2017-02-25,MGM Home Entertainment,Rotten,33.0,24.0,Spilled,21.0,1813.0,7.0,8.0,16.0,de063f3f-2d35-4e1c-8636-6eb4c16bd236
4,4,Alibi,1928/29,Feature Productions,Nominee,1929,91,"Action,Crime,Romance",5.8,765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,609887c2-877c-43a4-b88c-e40e31096a98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,566,Mank,2020,"Ceán Chaffin, Eric Roth, and Douglas Urbanski",Nominee,2020,131,"Biography,Comedy,Drama",6.9,"65,380",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47d4ae4f-e782-4cd9-9508-4a07302b1c1a
567,567,Minari,2020,Christina Oh,Nominee,2020,115,Drama,7.5,"57,976",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7262b3a8-214d-4205-985c-70e0860f3236
568,568,Promising Young Woman,2020,"Ben Browning, Ashley Fox, Emerald Fennell, and...",Nominee,2020,113,"Crime,Drama,Thriller",7.5,"122,269",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d64c669b-7a73-496a-bddb-19cb09264371
569,569,Sound of Metal,2020,Bert Hamelinck and Sacha Ben Harroche,Nominee,2019,120,"Drama,Music",7.8,"102,807",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,647357e9-c067-46bd-aaeb-24d4344ec124


In [39]:
# drop unwanted columns
new_df=oscars_df.drop(columns=['Movie Info', 'Genres', 'Critic Consensus', 'Film ID', 'Audience Status', 'Tomatometer Status', 'Authors', 'Streaming Release Date', 'Original Release Date', 'Oscar Year', 'Tomatometer Rating','Tomatometer Count','Audience Rating','Audience Count','Tomatometer Top Critics Count','Tomatometer Fresh Critics Count','Tomatometer Rotten Critics Count'], axis=1)
new_df.tail(20)

,Unnamed: 0,Film,Film Studio/Producer(s),Award,Year of Release,Movie Time,Movie Genre,IMDB Rating,IMDB Votes,Content Rating,Directors,Actors,Production Company
551,551,Roma,Gabriela Rodríguez and Alfonso Cuarón,Nominee,2018,135,Drama,7.7,"147,705",R,Alfonso Cuarón,"Yalitza Aparicio, Marina de Tavira, Diego Cort...",Netflix
552,552,A Star Is Born,"Bill Gerber, Bradley Cooper, and Lynette Howel...",Nominee,2018,136,"Drama,Music,Romance",7.6,"354,157",R,Bradley Cooper,"Lady Gaga, Bradley Cooper, Sam Elliott, Andrew...",Warner Bros. Pictures
553,553,Vice,"Dede Gardner, Jeremy Kleiner, Adam McKay, and ...",Nominee,2018,132,"Biography,Comdey,Drama",7.2,"132,014",R,Adam McKay,"Christian Bale, Amy Adams, Steve Carell, Sam R...",Annapurna Pictures
554,554,Parasite,Kwak Sin-ae and Bong Joon-ho,Winner,2019,132,"Comedy,Drama,Thriller",8.6,"651,517",NaN,NaN,NaN,NaN
555,555,1917,"Sam Mendes, Pippa Harris, Jayne-Ann Tenggren, ...",Nominee,2019,119,"Action,Drama,War",8.3,"485,648",NaN,NaN,NaN,NaN
556,556,Ford v Ferrari,"Peter Chernin, Jenno Topping, and James Mangold",Nominee,2019,152,"Action,Biography,Drama",8.1,"331,665",NaN,NaN,NaN,NaN
557,557,The Irishman,"Martin Scorsese, Robert De Niro, Jane Rosentha...",Nominee,2019,209,"Biography,Crime,Drama",7.8,"349,303",R,Martin Scorsese,"Robert De Niro, Al Pacino, Joe Pesci, Jesse Pl...",Netflix
558,558,Jojo Rabbit,"Carthew Neal, Taika Waititi, and Chelsea Winst...",Nominee,2019,108,"Comedy,Drama,War",7.9,"335,856",NaN,NaN,NaN,NaN
559,559,Joker,"Todd Phillips, Bradley Cooper, and Emma Tillin...",Nominee,2019,122,"Crime,Drama,Thriller",8.4,"1,056,051",R,Todd Phillips,"Joaquin Phoenix, Robert De Niro, Zazie Beetz, ...",Warner Bros. Pictures
560,560,Little Women,Amy Pascal,Nominee,2019,135,"Drama,Romance",7.8,"163,410",NaN,NaN,NaN,NaN


In [40]:
final_df=new_df.fillna("unknown")
final_df.tail(20)

,Unnamed: 0,Film,Film Studio/Producer(s),Award,Year of Release,Movie Time,Movie Genre,IMDB Rating,IMDB Votes,Content Rating,Directors,Actors,Production Company
551,551,Roma,Gabriela Rodríguez and Alfonso Cuarón,Nominee,2018,135,Drama,7.7,"147,705",R,Alfonso Cuarón,"Yalitza Aparicio, Marina de Tavira, Diego Cort...",Netflix
552,552,A Star Is Born,"Bill Gerber, Bradley Cooper, and Lynette Howel...",Nominee,2018,136,"Drama,Music,Romance",7.6,"354,157",R,Bradley Cooper,"Lady Gaga, Bradley Cooper, Sam Elliott, Andrew...",Warner Bros. Pictures
553,553,Vice,"Dede Gardner, Jeremy Kleiner, Adam McKay, and ...",Nominee,2018,132,"Biography,Comdey,Drama",7.2,"132,014",R,Adam McKay,"Christian Bale, Amy Adams, Steve Carell, Sam R...",Annapurna Pictures
554,554,Parasite,Kwak Sin-ae and Bong Joon-ho,Winner,2019,132,"Comedy,Drama,Thriller",8.6,"651,517",unknown,unknown,unknown,unknown
555,555,1917,"Sam Mendes, Pippa Harris, Jayne-Ann Tenggren, ...",Nominee,2019,119,"Action,Drama,War",8.3,"485,648",unknown,unknown,unknown,unknown
556,556,Ford v Ferrari,"Peter Chernin, Jenno Topping, and James Mangold",Nominee,2019,152,"Action,Biography,Drama",8.1,"331,665",unknown,unknown,unknown,unknown
557,557,The Irishman,"Martin Scorsese, Robert De Niro, Jane Rosentha...",Nominee,2019,209,"Biography,Crime,Drama",7.8,"349,303",R,Martin Scorsese,"Robert De Niro, Al Pacino, Joe Pesci, Jesse Pl...",Netflix
558,558,Jojo Rabbit,"Carthew Neal, Taika Waititi, and Chelsea Winst...",Nominee,2019,108,"Comedy,Drama,War",7.9,"335,856",unknown,unknown,unknown,unknown
559,559,Joker,"Todd Phillips, Bradley Cooper, and Emma Tillin...",Nominee,2019,122,"Crime,Drama,Thriller",8.4,"1,056,051",R,Todd Phillips,"Joaquin Phoenix, Robert De Niro, Zazie Beetz, ...",Warner Bros. Pictures
560,560,Little Women,Amy Pascal,Nominee,2019,135,"Drama,Romance",7.8,"163,410",unknown,unknown,unknown,unknown


In [41]:
# Count the number of null values in each column
null_counts = final_df.isnull().sum()
# Display the results
print(null_counts)

Unnamed: 0                 0
Film                       0
Film Studio/Producer(s)    0
Award                      0
Year of Release            0
Movie Time                 0
Movie Genre                0
IMDB Rating                0
IMDB Votes                 0
Content Rating             0
Directors                  0
Actors                     0
Production Company         0
dtype: int64


In [42]:
final_df.to_csv("cleaned_oscar_df.csv", index=False)

In [30]:
# Read in data from csv
df=spark.read.csv("oscars_df.csv",header=True)
df.tail(10)
df.show()

+---+--------------------+----------+-----------------------+-------+---------------+----------+--------------------+-----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+----------------------+--------------------+--------------------+------------------+--------------------+--------------------+---------------+--------------+-----------------------------+-------------------------------+--------------------------------+--------------------+
|_c0|                Film|Oscar Year|Film Studio/Producer(s)|  Award|Year of Release|Movie Time|         Movie Genre|IMDB Rating|IMDB Votes|          Movie Info|              Genres|    Critic Consensus|      Content Rating|           Directors|             Authors|              Actors|Original Release Date|Streaming Release Date|  Production Company|  Tomatometer Status|Tomatometer Rating|   Tomatometer Count|   

In [34]:
new_df=df.drop('Movie Info', 'Genres', 'Critic Consensus', 'Film ID', 'Audience Status', 'Tomatometer Status', 'Authors', 'Streaming Release Date', 'Original Release Date', 'Oscar Year')
new_df.show()

+---+--------------------+-----------------------+-------+---------------+----------+--------------------+-----------+----------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+---------------+--------------+-----------------------------+-------------------------------+--------------------------------+
|_c0|                Film|Film Studio/Producer(s)|  Award|Year of Release|Movie Time|         Movie Genre|IMDB Rating|IMDB Votes|      Content Rating|           Directors|              Actors|  Production Company|Tomatometer Rating|   Tomatometer Count|Audience Rating|Audience Count|Tomatometer Top Critics Count|Tomatometer Fresh Critics Count|Tomatometer Rotten Critics Count|
+---+--------------------+-----------------------+-------+---------------+----------+--------------------+-----------+----------+--------------------+--------------------+--------------------+--------------------+------------------+--------